In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np
import math
import os
from io import StringIO

In [2]:
# retrieve via \copy (SELECT situs_address FROM corelogic_usda.current_tax_200627_latest_all_add_vars_add_progs_geom_blk WHERE geoid_cnty = '01073') TO '~/01073.csv' CSV header;
df = pd.read_csv('../data/temp/01073.csv')
df

,situs_address
0,825 CHEYENNE DR WARRIOR AL 351805900
1,837 CHEYENNE DR WARRIOR AL 35180
2,844 CHEYENNE DR WARRIOR AL 351805901
3,826 CHEYENNE DR WARRIOR AL 351805901
4,797 HART CIR WARRIOR AL 35180
...,...
198015,8757 SHADES FALLS WAY BESSEMER AL 35022
198016,8591 SERENE DR MC CALLA AL 351114230
198017,5600 BLACK CREEK DR BESSEMER AL 35022
198018,13931 ROCKHOUSE RD BROOKWOOD AL 35444


In [3]:
df = df.dropna()
df

,situs_address
0,825 CHEYENNE DR WARRIOR AL 351805900
1,837 CHEYENNE DR WARRIOR AL 35180
2,844 CHEYENNE DR WARRIOR AL 351805901
3,826 CHEYENNE DR WARRIOR AL 351805901
4,797 HART CIR WARRIOR AL 35180
...,...
198015,8757 SHADES FALLS WAY BESSEMER AL 35022
198016,8591 SERENE DR MC CALLA AL 351114230
198017,5600 BLACK CREEK DR BESSEMER AL 35022
198018,13931 ROCKHOUSE RD BROOKWOOD AL 35444


# Batch area processing

In [4]:
# headers: 1	4600 Silver Hill Road	Washington	DC	20233

In [5]:
bdf = pd.DataFrame()
bdf['street'] = df['situs_address'].apply(lambda x: x.split(' AL')[0])
bdf['county'] = 'Jefferson County'
bdf['state'] = 'AL'
bdf['zip'] = df['situs_address'].apply(lambda x: x.split('AL ')[-1])
bdf

,street,county,state,zip
0,825 CHEYENNE DR WARRIOR,Jefferson County,AL,351805900
1,837 CHEYENNE DR WARRIOR,Jefferson County,AL,35180
2,844 CHEYENNE DR WARRIOR,Jefferson County,AL,351805901
3,826 CHEYENNE DR WARRIOR,Jefferson County,AL,351805901
4,797 HART CIR WARRIOR,Jefferson County,AL,35180
...,...,...,...,...
198015,8757 SHADES FALLS WAY BESSEMER,Jefferson County,AL,35022
198016,8591 SERENE DR MC CALLA,Jefferson County,AL,351114230
198017,5600 BLACK CREEK DR BESSEMER,Jefferson County,AL,35022
198018,13931 ROCKHOUSE RD BROOKWOOD,Jefferson County,AL,35444


In [6]:
dfs = np.array_split(bdf, (len(bdf)/10000) + 1)

In [7]:
chunk_names = []
for i in range(len(dfs)):
    save_name = '../data/temp/01073_chunk_{i}.csv'.format(i = str(i).zfill(2))
    try:
        # print(i)
        dfs[i].to_csv(save_name)
    except:
        pass
    # print(os.path.isfile(save_name))
    if os.path.isfile(save_name):
        chunk_names.append(save_name)
print(len(chunk_names))

20


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
# https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
# https://curlconverter.com/
pbar = tqdm(chunk_names)
for chunk in pbar:
    save_name = '{chunk_name}_geocoded.csv'.format(chunk_name = chunk.split('.csv')[0])
    # print(save_name)
    pbar.set_description(save_name)
    if os.path.isfile(save_name): # skip if already downloaded
        continue
    files = {
        'addressFile': open(chunk, 'r'),
        'benchmark': (None, '2020'),
    }

    response = requests.post('https://geocoding.geo.census.gov/geocoder/locations/addressbatch', files=files)
    with open(save_name, 'wb') as f:
        f.write(response.content)

../data/temp/01073_chunk_19_geocoded.csv: 100%|████████████████████████████████████████████████████████| 20/20 [02:08<00:00,  6.44s/it]
